In [ ]:
# User defined working directory
work_dir = '/Users/Tem/Documents/naturalistic-threat-ptsd'

In [ ]:
# Import packages
import warnings
warnings.filterwarnings("ignore")
import os
import pandas as pd
import numpy as np
from nilearn import image
from nilearn import input_data
from datetime import datetime
from nltools.file_reader import onsets_to_dm
from nltools.data import Brain_Data, Design_Matrix
from nltools.stats import find_spikes 
from nltools.stats import zscore

In [ ]:
# Load list of subIDs 
cohort_IDs = pd.read_csv(f"{work_dir}/data/subs/cohort_IDs.csv") 
sub_list = cohort_IDs['subject'].tolist()
nCohort = len(sub_list)

## Smooth fMRI data

In [ ]:
# User defined variables
unsmoothed_dir = f"{work_dir}/data/neural-classical/niftis/classical-prepped-unsmoothed" 
smoothed_output_dir = f"{work_dir}/data/neural-classical/niftis/classical-prepped-smoothed-6mm" 
fwhm = 6

In [ ]:
# Smooth fMRI files
if os.path.exists(smoothed_output_dir) == False:
        os.makedirs(smoothed_output_dir)
        print(f"Created smoothed_output_dir: \n{smoothed_output_dir}")

unsmoothed_list = os.listdir(f"{unsmoothed_dir}")
unsmoothed_list = [x for x in unsmoothed_list if 'nii' in x]

for i in unsmoothed_list:
    subID = i.split('_')[0]
    startTime = datetime.now()
    unsmoothed_image = image.load_img(f"{unsmoothed_dir}/{i}")
    smoothed_image = image.smooth_img(unsmoothed_image, fwhm = fwhm)
    smoothed_image.to_filename(f"{smoothed_output_dir}/{i}")
    print(f"{subID} smoothing runtime = {datetime.now() - startTime}")

## Generate design matrices

In [ ]:
# User defined variables
smoothed_dir = f"{work_dir}/data/neural-classical/niftis/classical-prepped-smoothed-6mm"
confound_dir = f"{work_dir}/data/neural-classical/confounds/raw-confounds"
dm_output_dir = f"{work_dir}/data/neural-classical/confounds/dm-6mm"
outlier_cutoff = 3 #define outlier cutoff for despiking
TR = 1

In [ ]:
# Create list of nifti and confound filenames
exp_smoothed_dir = os.listdir(f"{smoothed_dir}")
exp_confound_dir = os.listdir(f"{confound_dir}")

smoothed_list = [None] * nCohort
confound_list = [None] * nCohort

for i in range(nCohort):
    smoothed_list[i] = [x for x in exp_smoothed_dir if sub_list[i] in x][0]
    confound_list[i] = [x for x in exp_confound_dir if sub_list[i] in x][0]

In [ ]:
# Load event files to generate condition regressors
conditions_dm = [None] * nCohort

for i in range(nCohort):
    confound_file = pd.read_csv(confound_dir + '/' + confound_list[i], delimiter = "\t")
    nTRs = len(confound_file)
    event_df = pd.read_csv(f"{work_dir}/data/task/events-classical/{sub_list[i]}.csv")
    event_df.columns = ['Stim', 'Onset', 'Duration']
    events_df_wide = onsets_to_dm(event_df, sampling_freq = 1/TR, run_length = nTRs)
    
    CSminus_cols = [col for col in events_df_wide.columns if 'minus' in col]
    CSplus_cols = [col for col in events_df_wide.columns if 'plus' in col and 'US' not in col]
    US_cols = [col for col in events_df_wide.columns if 'US' in col]
    
    cond_regressors = pd.DataFrame({'CSminus': events_df_wide[CSminus_cols].sum(axis = 1),
                                'CSplus': events_df_wide[CSplus_cols].sum(axis = 1),
                                'US': events_df_wide[US_cols].sum(axis = 1)})
    
    conditions_dm[i] = Design_Matrix(cond_regressors, sampling_freq = 1/TR).convolve()

In [ ]:
# Assemble full design matrices (with motion parameters) and save as tsv files 
if os.path.exists(dm_output_dir) == False:
    os.makedirs(dm_output_dir)
    print(f"Created dm_output_dir: \n{dm_output_dir}\n")
    
full_dms = [None] * nCohort

for i in range(nCohort):
    print(sub_list[i])
    
    # Create BrainData object
    print('creating braindata object...')
    startTime = datetime.now()
    smoothed_img = image.load_img(smoothed_dir + '/' + smoothed_list[i])
    bd_func_img = Brain_Data(smoothed_img)
    print(f"braindata runtime = {datetime.now() - startTime}\n")
    
    # Identify spikes in fmri data
    spikes = bd_func_img.find_spikes(global_spike_cutoff = outlier_cutoff, diff_spike_cutoff = outlier_cutoff)
    spikes = spikes.drop(labels = 'TR', axis = 1)
    
    # Assemble fMRI regressors
    fMRI_regressors = pd.read_csv(confound_dir + '/' + confound_list[i], delimiter = "\t")
    fMRI_regressors = fMRI_regressors[['global_signal', 'trans_x', 'trans_y', 'trans_z', 'rot_x', 'rot_y', 'rot_z']]
    fMRI_regressors['intercept'] = 1
    fMRI_regressors = Design_Matrix(pd.concat([fMRI_regressors, spikes], axis = 1), sampling_freq = 1/TR)
    
    # Assemble full design matrix
    dm = pd.concat([conditions_dm[i], fMRI_regressors], axis = 1)
    
    # Save design matrix 
    full_dms[i] = dm
    dm_output_file = f"{dm_output_dir}/{sub_list[i]}_design-mat.tsv"
    dm.to_csv(dm_output_file, sep = '\t', index = False)

## Mask fMRI data

In [ ]:
# User defined variables
mask_dir = f"{work_dir}/masks/amygdala-harvard-oxford"
dm_dir = f"{work_dir}/data/neural-classical/confounds/dm-6mm"
smoothed_dir = f"{work_dir}/data/neural-classical/niftis/classical-prepped-smoothed-6mm"
npy_output_dir = f"{work_dir}/data/neural-classical/masked/amygdala-harvard-oxford-6mm"
save_suffix = '_6mm' #suffix appended to npy fmri data arrays

In [ ]:
# Create a list of the ROIs to be masked 
ROI_mask_list = os.listdir(f"{mask_dir}")
ROI_mask_list = [x for x in ROI_mask_list if 'nii' in x]
nROIs = len(ROI_mask_list)

print(ROI_mask_list)

In [ ]:
# Create list of nifti filenames
exp_smoothed_dir = os.listdir(f"{smoothed_dir}")

smoothed_list = [None] * nCohort

for i in range(nCohort):
    smoothed_list[i] = [x for x in exp_smoothed_dir if sub_list[i] in x][0]

In [ ]:
# Generate masker objects
all_maskers = [None] * len(ROI_mask_list)

first_sub = sub_list[0]
func_file = f"{smoothed_dir}/{smoothed_list[0]}"
func_img = image.load_img(func_file)

for i in range(nROIs):
    mask_file = f"{mask_dir}/{ROI_mask_list[i]}"
    mask_img = image.load_img(mask_file)
    if mask_img.shape != func_img.shape[0:3]:
        mask_img = image.resample_to_img(mask_img, func_img, interpolation = 'nearest')
    all_maskers[i] = input_data.NiftiMasker(mask_img = mask_img, 
                                    mask_strategy = 'epi', 
                                    standardize = True,
                                    detrend = True,
                                    low_pass = 0.1,
                                    high_pass = 0.01,
                                    t_r = 1)
    
print(f"Generated {i + 1} maskers.")

In [ ]:
# Mask fMRI data for each subject using each mask and save masked data as npy arrays
for i in range(nCohort):
    func_file = image.load_img(smoothed_dir + '/' + smoothed_list[i])
    confound_file = f"{dm_dir}/{sub_list[i]}_design-mat.tsv"
    confound_df = pd.read_csv(confound_file, delimiter = "\t")
    
    for j in range(nROIs):
        cROI = ROI_mask_list[j][:-9]
        if os.path.exists(f"{npy_output_dir}/{cROI}_6mm/") == False:
            os.makedirs(f"{npy_output_dir}/{cROI}_6mm/")
        cData = all_maskers[j].fit_transform(func_file, confounds = confound_df)
        cData = np.transpose(cData, (1, 0))
        np.save(f"{npy_output_dir}/{cROI}_6mm/{sub_list[i]}_6mm", cData)
        
    print(f"done masking {sub_list[i]} data...")